In [54]:
import pandas as pd
import numpy as np
import os
import os.path as osp
from tqdm.notebook import tqdm
import sys
import matplotlib.pyplot as plt
sys.path.append('../scripts')
import featurizer
sys.path.append('../../../../scripts')
import preprocessing as preprocessing
import warnings

In [55]:
#df = preprocessing.load_df('../../../../../data/data_new/raw/Binance/BTC_USDT/BTC_USDT.csv')

In [56]:
def load_and_featurize(path:str):
    """
    Restrict to just the two day strategy
    """
    
    df = preprocessing.load_df(path)
    keys = df[(df.index.hour == 23) & (df.index.minute == 59)].index.tolist()
    
    e = {}

    for x in tqdm(keys):
        a = pd.Timedelta('5D') 
        b = pd.Timedelta('2D')
        c = pd.Timedelta('10m')
        X_chunk = df.loc[(df.index >= x-a) & (df.index <= x-c)]
        y_chunk = df.loc[(df.index >= x) & (df.index <= x+b)]
        e[x] = (X_chunk,y_chunk)
        
    mode1,mode2 = pd.Series([(len(e[k][0]),len(e[k][1])) for k in e.keys()]).value_counts().index[0]    
        
    # restrict to some of the data where we have same size data

    A = [((len(e[k][0]) == mode1) and (len(e[k][1]) == mode2)) for k in keys]

    print("We make use of {}% of the data".format(np.array(A).sum()*100/len(keys))) 
    
    
    featurized = {}

    for i,k in tqdm(enumerate(keys)):
        if A[i]:
            
            A0= featurize(e[k][0])
            B0= get_y(e[k][1])
            
            featurized[k] = (A0,B0)
        
    feat_keys = list(featurized.keys())
    feat_keys.sort()
    
    
    X = []
    y = []
    
    for f in feat_keys:
        X.append(featurized[f][0])
        y.append(featurized[f][1])
    
    tmp_dict = {}
    tmp_dict['X'] = X
    tmp_dict['y'] = y
    tmp_dict['timestamp'] = feat_keys
    
    return pd.DataFrame.from_dict(tmp_dict)

In [57]:
path = '../../../../../data/Binance/BTC_USDT.csv'

In [58]:
df = preprocessing.load_df(path)

Total NaN : 

timestamp    0
open         0
high         0
low          0
close        0
volume       0
returns      1
dtype: int64



In [59]:
#preprocessing.resample(df,'120min')

In [60]:
warnings.filterwarnings('ignore')



df = preprocessing.load_df(path)
keys = df[(df.index.hour == 23) & (df.index.minute == 59)].index.tolist()

e = {}

for x in tqdm(keys):
    a = pd.Timedelta('5D') 
    b = pd.Timedelta('2D')
    c = pd.Timedelta('10m')
    X_chunk = df.loc[(df.index >= x-a) & (df.index <= x-c)]
    y_chunk = df.loc[(df.index >= x) & (df.index <= x+b)]
    e[x] = (X_chunk,y_chunk)
    
mode1,mode2 = pd.Series([(len(e[k][0]),len(e[k][1])) for k in e.keys()]).value_counts().index[0]    
    
# restrict to some of the data where we have same size data

A = [((len(e[k][0]) == mode1) and (len(e[k][1]) == mode2)) for k in keys]

print("We make use of {}% of the data".format(np.array(A).sum()*100/len(keys))) 


featurized = {}

for i,k in tqdm(enumerate(keys)):
    if A[i]:
        
        A0= featurizer.featurize(e[k][0])
        B0= featurizer.get_y(e[k][1])
        
        featurized[k] = (A0,B0)
    
feat_keys = list(featurized.keys())
feat_keys.sort()


X = []
y = []

for f in feat_keys:
    X.append(featurized[f][0])
    y.append(featurized[f][1])

tmp_dict = {}
tmp_dict['X'] = X
tmp_dict['y'] = y
tmp_dict['timestamp'] = feat_keys



Total NaN : 

timestamp    0
open         0
high         0
low          0
close        0
volume       0
returns      1
dtype: int64



  0%|          | 0/1485 [00:00<?, ?it/s]

We make use of 94.07407407407408% of the data


0it [00:00, ?it/s]

In [61]:
len(keys),len(featurized.keys())

(1485, 1397)

In [62]:
fkeys = list(featurized.keys())
fkeys.sort()

lengths = pd.Series([len(featurized[k][0]) for k in fkeys])

In [63]:
lengths.value_counts()

124    1359
122      22
120       9
116       4
114       3
dtype: int64

In [11]:
bad_feat_keys = {}
for f in feat_keys:
    if len(featurized[f][0]) in bad_feat_keys.keys():
                  bad_feat_keys[len(featurized[f][0])].append(f)
    else:
        bad_feat_keys[len(featurized[f][0])] = [f]

In [31]:
aaa = bad_feat_keys[122][0]
bbb = bad_feat_keys[124][0]

In [28]:
a = e[bad_feat_keys[122][0]][0]
b = e[bad_feat_keys[124][0]][0]

In [20]:
a

,timestamp,open,high,low,close,volume,returns,dollar_volume
date,,,,,,,,
2018-10-14 23:59:00,1539561540000,6343.96,6343.96,6335.63,6339.34,19.324838,-0.000728,122551.358903
2018-10-15 00:00:00,1539561600000,6339.34,6342.77,6328.51,6330.00,15.305083,-0.001473,96952.650128
2018-10-15 00:01:00,1539561660000,6329.99,6333.67,6327.48,6330.78,9.991315,0.000123,63248.870606
2018-10-15 00:02:00,1539561720000,6327.75,6333.32,6327.50,6327.65,21.466550,-0.000494,135833.888435
2018-10-15 00:03:00,1539561780000,6328.05,6329.43,6320.00,6321.54,28.772304,-0.000966,181978.924478
...,...,...,...,...,...,...,...,...
2018-10-19 23:45:00,1539992700000,6539.47,6540.69,6538.60,6540.69,5.100679,0.000187,33358.848714
2018-10-19 23:46:00,1539992760000,6539.12,6540.70,6534.73,6537.22,10.663520,-0.000531,69719.906558
2018-10-19 23:47:00,1539992820000,6537.20,6539.65,6534.76,6538.87,2.171868,0.000252,14199.748999


In [21]:
tmp = preprocessing.resample(a,'120min')
    
    
a0 = tmp.returns.values.T.tolist()
a1 = tmp.dollar_volume.values.T.tolist()
a2 = [tmp.returns.std()] # risk measured over the data 
a3 = [tmp.dollar_volume.std()] 
a4 = [(-df.open.iloc[0] + df.high.max())/(df.open.iloc[0])]
a5 = [(df.high.max() - df.low.min())/(df.low.min())]

In [27]:
len(a0),len(a1),len(a2),len(a3),len(a4),len(a5)

(59, 59, 1, 1, 1, 1)

In [29]:
tmp = preprocessing.resample(b,'120min')
    
    
a0 = tmp.returns.values.T.tolist()
a1 = tmp.dollar_volume.values.T.tolist()
a2 = [tmp.returns.std()] # risk measured over the data 
a3 = [tmp.dollar_volume.std()] 
a4 = [(-df.open.iloc[0] + df.high.max())/(df.open.iloc[0])]
a5 = [(df.high.max() - df.low.min())/(df.low.min())]

In [30]:
len(a0),len(a1),len(a2),len(a3),len(a4),len(a5)

(60, 60, 1, 1, 1, 1)

In [41]:
e[aaa][0].drop_duplicates(subset = 'timestamp')

,timestamp,open,high,low,close,volume,returns,dollar_volume
date,,,,,,,,
2018-10-14 23:59:00,1539561540000,6343.96,6343.96,6335.63,6339.34,19.324838,-0.000728,122551.358903
2018-10-15 00:00:00,1539561600000,6339.34,6342.77,6328.51,6330.00,15.305083,-0.001473,96952.650128
2018-10-15 00:01:00,1539561660000,6329.99,6333.67,6327.48,6330.78,9.991315,0.000123,63248.870606
2018-10-15 00:02:00,1539561720000,6327.75,6333.32,6327.50,6327.65,21.466550,-0.000494,135833.888435
2018-10-15 00:03:00,1539561780000,6328.05,6329.43,6320.00,6321.54,28.772304,-0.000966,181978.924478
...,...,...,...,...,...,...,...,...
2018-10-19 23:45:00,1539992700000,6539.47,6540.69,6538.60,6540.69,5.100679,0.000187,33358.848714
2018-10-19 23:46:00,1539992760000,6539.12,6540.70,6534.73,6537.22,10.663520,-0.000531,69719.906558
2018-10-19 23:47:00,1539992820000,6537.20,6539.65,6534.76,6538.87,2.171868,0.000252,14199.748999


In [ ]:
for i,k in tqdm(enumerate(keys)):
    if A[i]:
        
        A0= featurize(e[k][0])
        B0= get_y(e[k][1])
        
        featurized[k] = (A0,B0)
    
feat_keys = list(featurized.keys())
feat_keys.sort()


X = []
y = []

for f in feat_keys:
    X.append(featurized[f][0])
    y.append(featurized[f][1])

tmp_dict = {}
tmp_dict['X'] = X
tmp_dict['y'] = y
tmp_dict['timestamp'] = feat_keys

In [8]:
os.listdir('../scripts')

['featurizer.py', '__pycache__', '.ipynb_checkpoints']

In [9]:
print(dir(preprocessing))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'check_for_missing_timestamps', 'data_preprocessing', 'datetime', 'go', 'load_df', 'math', 'mode', 'np', 'os', 'pd', 'plt', 'px', 'resample', 'statistics', 'sys', 'tqdm']
